Avaliação Nº2 - 4/10/22

Aluno: Murilo Luis Calvo Neves

RA: 129037

### Exercício 1

Heurística: 

    1) Pegar os pontos A B C e D do usuário
    
    2) Montar os vetores AB e CD (Retas: A + tAB; C + uCD)
        
    3) Encontrar os valores que resolvam o sistema de equações A + tAB = C + uCD, para t e u.
    
    4) Com estes valores calculados, ver se eles estão no intervalo [0,1], caso estejam, achamos um ponto de interseção entre os segmentos.
    
    5) Caso algum deles não esteja, não há interseção entre os segmentos de reta.

In [7]:
def vetorAPartirDePontos(A: list, B: list) -> list:
    # Cria um vetor a partir de dois pontos
    resultado = []
    for i in range(len(A)):
        resultado.append(B[i]-A[i])
    return resultado
    
def soma(*argv) -> list:
    # Função que soma n vetores
    soma = []
    for vetor in argv:
        for dimensao, elemento in enumerate(vetor):
            if dimensao + 1 <= len(soma): 
                soma[dimensao] += elemento
            else:
                soma.append(elemento)
    return soma

def multPorNumero(v: list, n: float) -> list:
    # Multiplica um vetor por numero escalar
    return [(n * i) for i in v]

def pegarNumero(tipo = 'int') -> float:
    # Pega um número do usuário, continua rodando até um número válido ser inserido
    while True:
        try:
            if tipo == 'float':
                x = float(input(">> "))
            else:
                x = int(input(">> "))
            return x
        except ValueError:
            print("Numero invalido")
            continue

def pegarPonto(nome: str, dimensoes: int) -> list:
    # Pega um ponto do usuário
    pto = []
    for dimensao in range(dimensoes):
        print(f"Insira o valor da dimensão {dimensao + 1} para o ponto {nome}: ")
        num = pegarNumero('float')
        pto.append(num)
    return pto
            
def main():
    # Pegando os valores do usuário
    print("Insira o numero de dimensões: ")
    dimensoes = pegarNumero('int')
    
    if dimensoes <= 1:
        print("Numero invalido de dimensões (deve ser ao menos 2)")
        return
    
    A = pegarPonto('A', dimensoes)
    B = pegarPonto('B', dimensoes)
    C = pegarPonto('C', dimensoes)
    D = pegarPonto('D', dimensoes)
    
    print(f"A: {A}\nB: {B}\nC: {C}\nD: {D}")
    
    AB = vetorAPartirDePontos(A,B)
    CD = vetorAPartirDePontos(C,D)
    
    print(f"AB: {AB}\nCD: {CD}")
    
    # Resolvendo as equações
    t,u = var('t','u')
    eqs = []
    
    for i in range(dimensoes):
        eqn = A[i] + t * AB[i] == C[i] + u * CD[i]
        eqs.append(eqn)

    solucoes = solve(eqs, [t,u])
    
    # Caso de não interseção de retas
    if len(solucoes) == 0:
        print("Não há interseção")
        return
    
    # Casos de não interseção de segmentos (parametrização no intervalo [0,1])
    if (solucoes[0][0].rhs() < 0 or solucoes[0][0].rhs() > 1):
        print("Não há interseção")
        return
    
    if (solucoes[0][1].rhs() < 0 or solucoes[0][1].rhs() > 1):
        print("Não há interseção")
        return
    
    # Obtendo coordenadas finais
    parteVetor = multPorNumero(AB, solucoes[0][0].rhs())
    
    coordenadasIntersecao = soma(A, parteVetor)
    
    if 'r' in str(coordenadasIntersecao[0]) or 'r' in str(coordenadasIntersecao[1]):
        print('Não há interseção (ou ela é um intervalo)')
        return
    
    print(f"As coordenadas de interseção são {coordenadasIntersecao}")
    
main()
    

Insira o numero de dimensões: 
>> 2
Insira o valor da dimensão 1 para o ponto A: 
>> 0
Insira o valor da dimensão 2 para o ponto A: 
>> 15
Insira o valor da dimensão 1 para o ponto B: 
>> 15
Insira o valor da dimensão 2 para o ponto B: 
>> 0
Insira o valor da dimensão 1 para o ponto C: 
>> -20
Insira o valor da dimensão 2 para o ponto C: 
>> -30
Insira o valor da dimensão 1 para o ponto D: 
>> 30
Insira o valor da dimensão 2 para o ponto D: 
>> 20
A: [0.0, 15.0]
B: [15.0, 0.0]
C: [-20.0, -30.0]
D: [30.0, 20.0]
AB: [15.0, -15.0]
CD: [50.0, 50.0]
As coordenadas de interseção são [12.5, 2.5]


### Exercício 2

Heurística:

    1) Pegar as coordenadas do ponto e a equação geral do plano
    
    2) Pegar o vetor ortogonal ao plano (dado pela eq. geral, (a,b,c)) e juntar ao ponto P dado para montar uma reta.
       
    3) Com essa reta montada, montar sua parametrização e mostrar ao usuário.

In [4]:
def pegarNumero(tipo = 'int') -> float:
    # Pega um número do usuário, continua rodando até um número válido ser inserido
    while True:
        try:
            if tipo == 'float':
                x = float(input(">> "))
            else:
                x = int(input(">> "))
            return x
        except ValueError:
            print("Numero invalido")
            continue

def pegarPonto(nome: str, dimensoes: int) -> list:
    # Pega um ponto do usuário
    pto = []
    for dimensao in range(dimensoes):
        print(f"Insira o valor da dimensão {dimensao + 1} para o ponto {nome}: ")
        num = pegarNumero('float')
        pto.append(num)
    return pto

def pegarPlano(nome: str, dimensoes: int) -> list:
    # Pega um plano do usuário (equação geral)
    coefs = []
    for dimensao in range(dimensoes + 1):
        print(f"Insira o valor do coeficiente {dimensao + 1} para o plano {nome}: ")
        num = pegarNumero('float')
        coefs.append(num)
    return coefs

def tryLoop(valorStr: str, variavel: str) -> float:
    # Tenta pegar o valor float atrelado a uma variável
    # Ex: '7x' = 7  '-9z' = -9   '-y' = -1
    valorStr = valorStr.replace(variavel, '')
    if valorStr == '':
        valorStr = 1
    if valorStr == '-':
        valorStr = -1

    try:
        return float(valorStr)
    except ValueError:
        return 0

def removerDesnecessario(parteCoefs: list) -> list:
    # Remove os valores desnecessários da lista com os coeficientes não processados da equação geral
    equacaoProcessada = []
    finalizado = False
    for i in range(len(parteCoefs)):
        if parteCoefs[i] == '' or   finalizado:
            continue
        if '=' in parteCoefs[i]:
            equacaoProcessada.append(parteCoefs[i].split('=')[0])
            finalizado = True
            continue
        else:
            equacaoProcessada.append(parteCoefs[i])
    return equacaoProcessada

def pegarCoeficientesEqGeral(entrada: str) -> list:
    # Pega os coeficientes de um texto contento uma equação geral de plano
    # ax + by + cz + d = 0
    entrada = entrada.replace('*', '').replace(' ', '').replace('-', '+-')
    valores = entrada.split('+')
    valores = removerDesnecessario(valores)
    x_coef = 0
    y_coef = 0
    z_coef = 0
    d_coef = 0

    for val in valores:
        try:
            d_coef += float(val)
        except ValueError:
            x_coef += tryLoop(val, 'x')
            y_coef += tryLoop(val, 'y')
            z_coef += tryLoop(val, 'z')

    coefs = [x_coef, y_coef, z_coef, d_coef]
    print(f"\nCoeficientes do plano: {coefs}")
    return coefs


def main():
    # Pegando valores do usuário
    print("Insira o numero de dimensoes: ")
    dimensoes = pegarNumero()
    
    if dimensoes <= 0:
        print("Numero de dimensoes invalido")
        return
    
    if dimensoes == 3:
        eq = input('Insira a equação geral do plano (ex: 7x - 9y - z + 2 = 0)')
        plano = pegarCoeficientesEqGeral(eq)
    else:
        plano = pegarPlano('O', dimensoes)
        
    ponto = pegarPonto('P', dimensoes)
    print(f"Seja R uma reta que passa por P = {ponto} e é ortogonal ao plano cujos coeficientes são O = {plano}")
    print("As equações paramétricas da reta R são:")
    
    # Mostrando as equações paramétricas ao usuário
    for i in range(dimensoes):
        if plano[i] > 0:
            print(f"| a{i+1} = {ponto[i]} + {plano[i]}t")
        if plano[i] < 0:
            print(f"| a{i+1} = {ponto[i]} - {-1*plano[i]}t")
        if plano[i] == 0:
            print(f"| a{i+1} = {ponto[i]}")
        
main()

Insira o numero de dimensoes: 
>> 3
Insira a equação geral do plano (ex: 7x - 9y - z + 2 = 0)7x-9y-z+2=0

Coeficientes do plano: [7.0, -9.0, -1.0, 2.0]
Insira o valor da dimensão 1 para o ponto P: 
>> 2
Insira o valor da dimensão 2 para o ponto P: 
>> 7
Insira o valor da dimensão 3 para o ponto P: 
>> 3
Seja R uma reta que passa por P = [2.0, 7.0, 3.0] e é ortogonal ao plano cujos coeficientes são O = [7.0, -9.0, -1.0, 2.0]
As equações paramétricas da reta R são:
| a1 = 2.0 + 7.0t
| a2 = 7.0 - 9.0t
| a3 = 3.0 - 1.0t


### Exercício 3

Heurística:

    1) Obter a equação vetorial da reta e a equação geral do plano
    
    2) Caso o vetor diretor da reta seja ortogonal ao vetor ortogonal ao plano (i.e, paralela ao plano):
        
        2a) Se o ponto de origem da reta pertencer ao plano, dizer que a reta está contida no plano e finalizar
        
        2b) Se o ponto não estiver no plano, dizer que a reta é paralela e finalizar
    
    3) Caso não sejam, determinar t tal que o ponto P = A + tV esteja no plano, i.e, substituindo P na equação geral do plano e resolvendo.

In [9]:
def pegarNumero(tipo = 'int') -> float:
    # Pega um número do usuário, continua rodando até um número válido ser inserido
    while True:
        try:
            if tipo == 'float':
                x = float(input(">> "))
            else:
                x = int(input(">> "))
            return x
        except ValueError:
            print("Numero invalido")
            continue

def soma(*argv) -> list:
    # Função que soma n vetores
    soma = []
    for vetor in argv:
        for dimensao, elemento in enumerate(vetor):
            if dimensao + 1 <= len(soma): 
                soma[dimensao] += elemento
            else:
                soma.append(elemento)
    return soma

def multPorNumero(v: list, n: float) -> list:
    # Multiplica um vetor por numero escalar
    return [(n * i) for i in v]
            
def pontoPertenceAoPlano(P: list, O: list) -> bool:
    resultado = 0
    for i in range(len(P)):
        resultado += P[i] * O[i]
    if resultado + O[-1] == 0:
        return True
    return False

def produtoEscalar(A: list, B: list) -> float:
    resultado = 0
    for i in range(len(A)):
        resultado += (A[i] * B[i])
    return resultado

def pegarPontoOuVetor(nome: str, dimensoes: int) -> list:
    # Pega um ponto ou vetor do usuário (são o mesmo tipo de objeto, só muda o nome)
    pto = []
    for dimensao in range(dimensoes):
        print(f"Insira o valor da dimensão {dimensao + 1} para o {nome}: ")
        num = pegarNumero('float')
        pto.append(num)
    return pto

def pegarPlano(nome: str, dimensoes: int) -> list:
    # Pega um plano do usuário (equação geral)
    coefs = []
    for dimensao in range(dimensoes + 1):
        print(f"Insira o valor do coeficiente {dimensao + 1} para o plano {nome}: ")
        num = pegarNumero('float')
        coefs.append(num)
    return coefs

def main():
    # Obtendo valores do usuário
    print("Insira o número de dimensões: ");
    dimensoes = pegarNumero('int')
    if dimensoes <= 0:
        print("Número inválido de dimensões")
        return
    print("Insira o ponto P da reta, o vetor diretor V da reta e os coeficientes do plano O: ")
    P = pegarPontoOuVetor('ponto P', dimensoes)
    V = pegarPontoOuVetor('vetor V', dimensoes)
    O = pegarPlano('O', dimensoes)
    
    vetorOrtogonalPlano = O[:-1]
    
    # Caso de paralelismo
    if produtoEscalar(V, vetorOrtogonalPlano) == 0:
        if pontoPertenceAoPlano(P, O):
            print('A reta está contida no plano')
            return
        else:
            print('A reta é paralela ao plano')
            return
    
    # Resolvendo equações para achar a interseção
    t = var('t')
    
    ptoFinal = []  # Vira uma lista com as equações paramétricas da reta que descrevem um ponto dependente em t
    for i in range(len(P)):
        ptoFinal.append(P[i] + t * V[i])
    
    # Substituindo ponto na equação do plano -> ax + by + cz + d = 0
    # x,y e z são as coordenadas da variável ptoFinal
    # a,b,c e d são os coeficientes da equação de plano
    
    eqs = O[-1]  # Já começamos somando d, que não é multiplicado por nenhuma coordenada do ponto
    for i in range(len(ptoFinal)):
        eqs += ptoFinal[i] * O[i]
    
    resultado = solve([eqs == 0], t)
    
    # Obtendo o ponto final
    coordsIntersecao = soma(P, multPorNumero(V, resultado[0].rhs()))
    
    print(f"As coordenadas de interseção entre a reta e o plano são: {coordsIntersecao}")
    
main()

Insira o número de dimensões: 
>> 3
Insira o ponto P da reta, o vetor diretor V da reta e os coeficientes do plano O: 
Insira o valor da dimensão 1 para o ponto P: 
>> 10
Insira o valor da dimensão 2 para o ponto P: 
>> -8
Insira o valor da dimensão 3 para o ponto P: 
>> 19
Insira o valor da dimensão 1 para o vetor V: 
>> -5
Insira o valor da dimensão 2 para o vetor V: 
>> 2
Insira o valor da dimensão 3 para o vetor V: 
>> 4
Insira o valor do coeficiente 1 para o plano O: 
>> -2
Insira o valor do coeficiente 2 para o plano O: 
>> 7
Insira o valor do coeficiente 3 para o plano O: 
>> -30
Insira o valor do coeficiente 4 para o plano O: 
>> 5
As coordenadas de interseção entre a reta e o plano são: [43.385416666666664, -21.354166666666664, -7.708333333333332]
